In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Business Understanding:
The ability to predict financial distress is crucial in assessing the creditworthiness of loan applicants. By identifying individuals at an increased risk of financial hardship within the next two years, lenders can make more informed decisions about loan eligibility and terms. This not only safeguards the financial health of the lending institution but also protects borrowers from taking on unsustainable debt burdens.

# Problem Statement:
Develop a high-accuracy, explainable credit scoring system to predict the likelihood of loan default for [target audience], utilizing historical lending data and advanced classification algorithms.  The system should identify key factors influencing creditworthiness and provide clear justifications for its decisions.


# Loading datasets

In [ ]:
train_df = pd.read_csv("/kaggle/input/GiveMeSomeCredit/cs-training.csv")
test_df = pd.read_csv("/kaggle/input/GiveMeSomeCredit/cs-test.csv")
sample_entry = pd.read_csv("/kaggle/input/GiveMeSomeCredit/sampleEntry.csv")

train_df.head()

# Data cleaning and pre-processing

In [ ]:
train_df.drop(columns = ['Unnamed: 0'], inplace = True)
train_df

# Exploratory Data Analysis
**Statistical summary of training dataset**

In [ ]:
train_df.describe()

In [ ]:
test_df

In [ ]:
test_df.drop(columns = ['Unnamed: 0'], inplace = True)
test_df

**Statistical summary of test dataset**

In [ ]:
test_df.describe()

In [ ]:
sample_entry

In [ ]:
train_df['SeriousDlqin2yrs'].value_counts()

In [ ]:
train_df.isna().sum()

In [ ]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy='mean').fit(train_df[['MonthlyIncome']])

has_monthly_income_na = train_df['MonthlyIncome'].isna()
train_df['MonthlyIncome'] = si.transform(train_df[['MonthlyIncome']])

train_df.loc[has_monthly_income_na, 'DebtRatio'] = train_df.loc[has_monthly_income_na, 'DebtRatio']/train_df.loc[has_monthly_income_na, 'MonthlyIncome']

In [ ]:
import seaborn as sns

sns.boxplot(train_df['DebtRatio'])
plt.show()

In [ ]:
def remove_outliers(df, column_name, iqr_multiplier = 2):
    # TODO check this iqr multiplier default value if it is too stringent
    q1 = df[column_name].quantile(0.25)
    q3 = df[column_name].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - iqr_multiplier * iqr
    upper_bound = q3 + iqr_multiplier * iqr
    df = df[(df[column_name] >= lower_bound) & \
            (df[column_name] <= upper_bound)]
    return df

train_df = remove_outliers(train_df, 'DebtRatio')

In [ ]:
sns.boxplot(train_df['DebtRatio'])
plt.show()

In [ ]:
sns.boxplot(train_df['MonthlyIncome'])
plt.show()

In [ ]:
train_df = remove_outliers(train_df, 'MonthlyIncome')

In [ ]:
sns.boxplot(train_df['MonthlyIncome'])
plt.show()

**Observe that the number of 0's are lot more than the 1's, training with such distribution had lead me to lesser accuracy therefore, we will only take fraction of data whose label is 0 this time.**

In [ ]:
independent_cols = ['RevolvingUtilizationOfUnsecuredLines', 'age',
                    'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
                    'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
                    'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
                    'NumberOfDependents']
dependent_col = 'SeriousDlqin2yrs'

In [ ]:
train_df[independent_cols].isna().sum()

In [ ]:
# All imports go here
import pandas as pd
import numpy as np
import random
random.seed(4)

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
numeric_preprocessor = Pipeline(
    steps=[
        ("imputation_mean", SimpleImputer(missing_values=np.nan, strategy="mean")),
        ("scaler", StandardScaler()),
    ]
)

preprocessor = ColumnTransformer(
    [
        ("numerical", numeric_preprocessor, independent_cols),
    ]
)
preprocessor

In [ ]:
df_sample = train_df.sample(5000, random_state=4)
X = df_sample[independent_cols]
y = df_sample[dependent_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 10)

In [ ]:
X_train.head(), y_train.head()

In [ ]:
rf = RandomForestClassifier(random_state=4)
lr = LogisticRegression(random_state=4)
models_to_try = [rf, lr]

In [ ]:
all_pipelines = {}
for m in models_to_try:
    # Create a scikit-learn pipeline
    all_pipelines[m] = pipeline = make_pipeline(preprocessor, m)
    # Perform cross-validation on the training data
    cross_val_scores = cross_val_score(pipeline, X_train, y_train, cv=4, scoring='roc_auc')
    print(m)
    print(f"Cross-Validation Scores: {cross_val_scores}")
    print(f"Mean Cross-Validation Score: {np.mean(cross_val_scores)}\n")

In [ ]:
# As seen above the best model is RandomForestRegressor
# Fit the RF model on training data
all_pipelines[rf].fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score

# Use only selected model to predict on the test data
y_pred = all_pipelines[rf].predict_proba(X_test)[:, 1]
print(f'Test AUC of best model = {roc_auc_score(y_test, y_pred)}')

In [ ]:
all_pipelines[rf].fit(X, y)

In [ ]:
test_df

In [ ]:
has_monthly_income_na = test_df['MonthlyIncome'].isna()
test_df['MonthlyIncome'] = si.transform(test_df[['MonthlyIncome']])

test_df.loc[has_monthly_income_na, 'DebtRatio'] = test_df.loc[has_monthly_income_na, 'DebtRatio']/test_df.loc[has_monthly_income_na, 'MonthlyIncome']

In [ ]:
test_df.isna().sum()

In [ ]:
test_df[dependent_col] = all_pipelines[rf].predict(test_df)

In [ ]:
test_df['SeriousDlqin2yrs'].value_counts(normalize=True)

In [ ]:
sample_entry

In [ ]:
sample_entry['Probability'] = all_pipelines[rf].predict_proba(test_df)[:, 1]

In [ ]:
sample_entry.to_csv('sampleEntry.csv', index=False)